<a href="https://colab.research.google.com/github/DanielFerreira3900/PremierLeague/blob/main/Clubes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Web Scrapping dos times


In [ ]:
mport requests
from bs4 import BeautifulSoup
import pandas as pd

# Dicionário com clubes e URLs
times_url = {
    "Manchester City": "https://www.transfermarkt.com/manchester-city/startseite/verein/281/saison_id/2024",
    "Arsenal": "https://www.transfermarkt.com/fc-arsenal/startseite/verein/11/saison_id/2024",
    "Liverpool": "https://www.transfermarkt.com/fc-liverpool/startseite/verein/31/saison_id/2024",
    "Aston Villa": "https://www.transfermarkt.com/aston-villa/startseite/verein/405/saison_id/2024",
    "Tottenham": "https://www.transfermarkt.com/tottenham-hotspur/startseite/verein/148/saison_id/2024",
    "Manchester United": "https://www.transfermarkt.com/manchester-united/startseite/verein/985/saison_id/2024",
    "Newcastle United": "https://www.transfermarkt.com/newcastle-united/startseite/verein/762/saison_id/2024",
    "West Ham": "https://www.transfermarkt.com/west-ham-united/startseite/verein/379/saison_id/2024",
    "Chelsea": "https://www.transfermarkt.com/fc-chelsea/startseite/verein/631/saison_id/2024",
    "Wolverhampton": "https://www.transfermarkt.com/wolverhampton-wanderers/startseite/verein/543/saison_id/2024",
    "Brighton": "https://www.transfermarkt.com/brighton-amp-hove-albion/startseite/verein/1237/saison_id/2024",
    "Fulham": "https://www.transfermarkt.com/fc-fulham/startseite/verein/931/saison_id/2024",
    "Bournemouth": "https://www.transfermarkt.com/afc-bournemouth/startseite/verein/989/saison_id/2024",
    "Crystal Palace": "https://www.transfermarkt.com/crystal-palace/startseite/verein/873/saison_id/2024",
    "Brentford": "https://www.transfermarkt.com/brentford-fc/startseite/verein/1148/saison_id/2024",
    "Everton": "https://www.transfermarkt.com/fc-everton/startseite/verein/29/saison_id/2024",
    "Nottingham Forest": "https://www.transfermarkt.com/nottingham-forest/startseite/verein/703/saison_id/2024",
    "Luton Town": "https://www.transfermarkt.com/luton-town/startseite/verein/1031/saison_id/2024",
    "Burnley": "https://www.transfermarkt.com/fc-burnley/startseite/verein/1132/saison_id/2024",
    "Sheffield United": "https://www.transfermarkt.com/sheffield-united/startseite/verein/350/saison_id/2024"
}

headers = {
    'User-Agent': 'Mozilla/5.0'
}

# Funções auxiliares
def converter_valor_mercado(valor):
    try:
        valor = valor.lower().replace('€', '').replace('milhões', ' mi').replace('mil', ' mil')
        if 'mil' in valor:
            return float(valor.replace(' mil', '').replace(',', '.')) * 1_000
        elif 'mi' in valor:
            return float(valor.replace(' mi', '').replace(',', '.')) * 1_000_000
        return None
    except:
        return None

def extrair_idade(texto):
    try:
        return texto.split(' ')[1].strip('()')
    except:
        return None

def extrair_ano_nascimento(texto):
    try:
        return texto.split(' ')[0]
    except:
        return None

# Coletar dados de todos os clubes
def coletar_dados():
    data = []
    for clube, url in times_url.items():
        print(f'🔎 Acessando {clube}...')
        try:
            response = requests.get(url, headers=headers)
            if response.status_code != 200:
                print(f'❌ Erro {response.status_code} ao acessar {url}')
                continue

            soup = BeautifulSoup(response.content, 'html.parser')
            tabela = soup.find('table', {'class': 'items'})
            if not tabela:
                print(f'❌ Tabela não encontrada para {clube}')
                continue

            linhas = tabela.find_all('tr', class_=['odd', 'even'])
            for linha in linhas:
                try:
                    nome = linha.find('td', {'class': 'hauptlink'}).text.strip()
                    posicao = linha.find_all('td')[4].text.strip()
                    idade_raw = linha.find_all('td')[5].text.strip()
                    valor_raw = linha.find('td', {'class': 'rechts hauptlink'}).text.strip()

                    idade = extrair_idade(idade_raw)
                    nascimento = extrair_ano_nascimento(idade_raw)
                    valor_convertido = converter_valor_mercado(valor_raw)

                    data.append([
                        nome, clube, posicao, idade, nascimento, valor_raw, valor_convertido
                    ])
                except Exception as e:
                    print(f'⚠️ Erro ao extrair jogador de {clube}: {e}')
        except Exception as erro:
            print(f'Erro de conexão com {clube}: {erro}')
    return data

# Execução principal
dados = coletar_dados()

# Criar DataFrame
df = pd.DataFrame(dados, columns=[
    'Nome do Jogador', 'Clube', 'Posição', 'Idade', 'Ano de Nascimento',
    'Valor de Mercado (texto)', 'Valor de Mercado (€)'
])

# Exportar
df.to_excel('dados_premier_league_2024.xlsx', index=False)
df.to_csv('dados_premier_league_2024.csv', index=False)
print("✅ Dados salvos com sucesso!")

 df.head()